### Intro

Topological sorting is a fun graph concept that can be applied to directed acyclic graphs (DAGs). To topologically sort a graph means to provide a linear sorting of the graph's vertices based on how the vertices are ordered in the DAG. Think of this like a dimensionality reduction technique that unravels a complicated network into one-dimention whose ordering preserves a minimal structure of the original network.

To give you an example, a classic application if topological sorting is to find an order in which to take college classes while factoring in course prerequisites. Let's say you have to take 20 classes to complete your major, but many of those classes have to be taken in sequences (e.g. single variable calculus -> multivariable calculus -> differential equations). These 20 classes can be arranged into a DAG network. Notice that there are often many different ways in which we can arrange a valid sequence of classes.

One algorithm that we can use to produce a topological sorting is called Kahn's algorithm. Kahn's algorithm leverages the concept of indegrees. The indegree of a particular vertex in a DAG is defined as the count of the number of vertices directly pointing at that vertex (e.g. the number of direct prerequisites a class has). Kahn's algorithm works by maintaining the current indegree of every vertex in the graph. At each step in the sort, we process the vertices with indegrees of 0, meaning they have no prerequisites or all of their prerequisites have previously been processed, and append them to the sorting. In processing these vertices, we also decrement the indegree of any neighbor vertex the current vertex serves as a prerequisite for, thereby allowing us to search for the next set of vertices with indegrees of 0. 

Often times, LeetCode graph problems will not provide you with a graph data structure but rather will give you an array of edges expecting you to build the graph from this array. The set of edges is structured such that each pair `[vertex_a, vertex_b]` represents a directed edge that points from `vertex_a` to `vertex_b`. Before we can run Kahn's algorithm, we need to build a graph data structure that tells us what vertices each vertex serves as a prerequisite for. There are many choices for building such a data structure but usually the best choice in this context is an adjacency list. For this tutorial, whenever we have to build a graph from a set of edges, we will build an adjacency list in the form of a dictionary where the keys will represent the vertex index, and the values will be either a list or a set of vertex indices that the key is a prerequisite of. For example, for the adjacency list `{0:[1,2,3], 1:[3]}`, vertex `0` is prerequisite of vertices `1`, `2`, and `3`.

### Template

In [ ]:
from collections import deque

def kahn(n: int, edges: list[list[int]]) -> list[int]:  # assume we're given the number of vertices and an array of directed edges
    adj_list = {i:[] for i in range(n)}                 # init adjacency list
    indegrees = {i:0 for i in range(n)}                 # init indegree counter
    for vertex_a, vertex_b in edges:                    # loop through each edge pair, build adj list and indegree counter
        adj_list[vertex_a].append(vertex_b)             # assume vertex_a directs to vertex_b
        indegrees[vertex_b] += 1

    zero_indegree_queue = deque([vertex for vertex, indegree in indegrees.items() if indegree == 0])  # init zero indegree queue    
    topological_sorted_order = []
    
    while zero_indegree_queue:                          # the rest of the algo is similar to the BFS pattern
        curr = zero_indegree_queue.popleft()            # pop a vertex with indegree 0
        topological_sorted_order.append(curr)           # since it has indegree 0, it is ready to be appended to the ans
        for nei in adj_list[curr]:                      # now loop through the neighbors the vertex directs to
            indegrees[nei] -= 1                         # decrement the neighbor's indegree count
            if indegrees[nei] == 0:                     # if the neighbor's indegree count is 0, it's ready to be added to the queue
                zero_indegree_queue.append(nei)
    
    # if the graph has a cycle, it's impossible to topologically sort, thus not all vertices will be visited, just return []
    return topological_sorted_order if len(topological_sorted_order) == n else []

### Examples

**[LC 1136. Parallel Courses](https://leetcode.com/problems/parallel-courses/) [Medium]**

This questions asks us to figure out the minimum number of semesters it takes to complete all the college courses, given a list of course prerequisites. A course may not be taken until all of its preqrequisite courses are taken in prior semesters. The input `n` indicates that there are `n` courses labeled from `1` to `n`, and the input `relations` is an array such that `relations[i] = [prevCoursei, nextCoursei]` indicates `prevCoursei` is a prerequisite to taking `nextCoursei`.

This question lends perfectly to our topological sorting template. We know we have `n` courses which we can think of as vertices in our graph, and we know we have an array `relations` which we can think of as a set of directed edges connecting our courses. To solve this problem, our goal will be to apply Kahn's algorithm to topologically sort the courses. We will make a small adjustment to our template as the goal of the problem is to find the number of semesters it takes to complete the topological sorting of courses. To track this, we will consider a semester to be the set of courses that currently have zero indegrees. For instance, the first semester we will process the courses that have no prerequisites at all. The second semester we will take the courses whose prerequisites were fulfilled by taking the first semester's set of courses. We repeat this loop until we run out of courses in our `zero_indegree_queue`.

Note: since we are just given the edges to the graph in this example, we must build a graph before we apply Kahn's algorithm so that we know for each course, which courses it serves as a prerequisite of. We'll build an adjacency list to accomplish this.

In [ ]:
def minimumSemesters(n: int, relations: list[list[int]]) -> int:   # n is number of courses, relations is an array of prereqs
    adj_list = {i:[] for i in range(1, n+1)}                       # init adjacency list
    indegrees = {i:0 for i in range(1, n+1)}                       # init indegree counter
    for course_a, course_b in relations:                           # loop through each prereq, build adj list and indegree counter
        adj_list[course_a].append(course_b)                        # note a is a prereq of b, so a -> b
        indegrees[course_b] += 1                                   # a is a prereq of b, so increment b's indegree count

    semester_cnt = 0                                               # init ans counter
    course_cnt = 0                                                 # track courses completed to compare at the end
    queue = deque([course for course, indegree in indegrees.items() if indegree == 0])  # courses with no prereqs, take these in 1st semester
    
    while queue:                                                   # each time this loop runs is a new semester
        for _ in range(len(queue)):                                # loop through the courses to be taken this semester
            curr_course = queue.popleft()                          # pop a course to simulate taking it
            course_cnt += 1
            for nei_course in adj_list[curr_course]:               # loop through the courses this course is a prereq of
                indegrees[nei_course] -= 1
                if indegrees[nei_course] == 0:                     # if neighbor course's prereqs all complete, take it next semester
                    queue.append(nei_course)
        semester_cnt += 1                                          # after completing this semester's courses, increment our ans

    # if we haven't completed all courses, there was a cycle, thus impossible to topologically sort, return -1
    return semester_cnt if course_cnt == n else -1

**[LC 269. Alien Dictionary](https://leetcode.com/problems/alien-dictionary/) [Hard]**

In this problem, we're given a list of words written in an alien language that uses the English alphabet. We don' know the correct order of the alien language letters (e.g. in English the order is `abcdef...`). We're told that the list of words is sorted lexicographically according to the sort order of the alien language's letters. Our task is to return a lexicographic sorting of the unique letters present in the list of words according to the alien language's rules. If it's impossible to do so we return an empty string.

Before we dive in lets understand the problem. We want to return a lexicographic sorting of the alien language letters, but just of the letters present in our list of words. The letters within a word don't tell us anything about the relative order, we'll only be using the ordering of the words to deduce the ording of the letters. We know such a sorting will be impossible if we find a cycle in the list of words (e.g. if we see both `a` sorted before `b` at one point but also `b` sorted before `a`). As an example, if we were provided with the words `[’bbb’, ‘aaacdef’]` one possible answer we could return is `'bacdef'`. Note that letters inconsequential to the sorting (in this example `'cdef'`) can be returned in any order amongst themselves (e.g. `'bafedc'` is also a valid sorting).

With this information we can start to see how topological sorting can help us. We know that if we take adjacent pairs of words in our list `words`, we can compare the words based on the first set of letters in which they differ to discover a sorting between that pair of letters. We can do this for every set of adjacent words to build out an adjacency list. Then we can run Kahn's algorithm. Really the only tricky part of this problem is building out the adjacency list.

In [ ]:
from collections import deque

def alienOrder(words: list[str]) -> str:
    adj_list = {l:[] for w in words for l in w}    # init for every unique letter in words since we need to sort every unique letter
    indegrees = {l:0 for w in words for l in w}    # init for every unique letter in words since we need to sort every unique letter
    for i in range(len(words)-1):                  # loop through every adjacent pair of words
        l = 0                                      # index of the letter that we'll use to compare adjacent words
        word_a, word_b = words[i], words[i+1]
        word_pair_min = min(len(word_a), len(word_b))
        while l < word_pair_min and word_a[l] == word_b[l]:  # equal letters don't tell us anything about the sorting so skip past those
            l += 1
        if l >= word_pair_min:                                                  # 2 cases if this triggers:
            if len(word_a) > len(word_b) and word_a[:word_pair_min] == word_b:  #   1: word_b is a prefix of word_a e.g. ['abc','ab']
                return ''                                                       #      but this violates lexicographic sorting rules
            continue                                                            #   2: word_a is a prefix of word_b which tells us nothing
        letter_a, letter_b = word_a[l], word_b[l]  # if no case triggers, we've found a letter sorting
        adj_list[letter_a].append(letter_b)        # letter_a is sorted before letter_b
        indegrees[letter_b] += 1                   # thus increment letter_b's indegree count

    queue = deque([letter for letter, indegree in indegrees.items() if indegree == 0])
    topological_sorted_order = ''

    while queue:                                   # run kahn's algorithm
        curr_letter = queue.popleft()              # pop a letter ready to be added to our sorting
        topological_sorted_order += curr_letter
        for nei_letter in adj_list[curr_letter]:   # loop through letters that we know follow curr_letter in the sorting
            indegrees[nei_letter] -= 1
            if indegrees[nei_letter] == 0:         # if we've already added all letters that precede nei_letter, add it to the queue
                queue.append(nei_letter)
    
    # if we have a cycle, there will be letters left with an indegree > 0, thus impossible to sort
    if [letter for letter, indegree in indegrees.items() if indegree > 0]:
        return ''
    return topological_sorted_order